# Check violations and Contingency test

In [2]:
#import external packages
import pandas as pd
import pandapower as pp
import numpy as np
import os,sys,inspect
import json


currentdir =  os.path.abspath(os.getcwd())
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)
from capacitymap.grid import grid
from capacitymap.analysis import analysis_check

In [4]:
#create a grid instance for testing
empt_df = pd.DataFrame(columns=['Start', 'End', 'Object_type','ObjectID', 'Status'])
net = pp.from_json(currentdir+'\data\svedala\svedala.json')
g = grid.Grid('test',net, {}, empt_df)

In [5]:
#counter to track number of power flows performed
analysis_check.check_violations.counter=0

#Perform pf and check results against limits
analysis_check.check_violations(g.grid, run_control = True,vmax=1.12, vmin=0.9, max_line_loading=105., max_trafo_loading=100., p_lim=1000.)

((False, False, False, False, False, False, False),
 [None, None, None, None, None, None, None])

In [6]:
#Contingency calculation
#Define lines and trafo to do N-1 on
line_to_test = list(g.grid.line.index)
trafo_to_test = list(g.grid.trafo.index)
contingency_scenario = [line_to_test, trafo_to_test]

# Limits for continency tests
contingency_limits = {'vmin': 0.87,
                'vmax': 1.15,
                'max_line_loading' : 105,
                'max_trafo_loading' : 100,
                'subscription_p_limits': 1000,
                'run_controllers': True}

#Reset counter to track number of power flows performed for 1 contingnency analysis
analysis_check.check_violations.counter=0

#Perform contingency analysis
cl,ct =analysis_check.contingency_test(g.grid,vmax=contingency_limits['vmax'],
                           vmin=contingency_limits['vmin'],
                           max_line_loading=contingency_limits['max_line_loading'],
                           max_trafo_loading=contingency_limits['max_trafo_loading'],
                           p_lim=contingency_limits['subscription_p_limits'],
                           run_control=contingency_limits['run_controllers'],
                           contingency_scenario=contingency_scenario)



In [7]:
#print results
print('Number of power flow: ', analysis_check.check_violations.counter)
print('Number of critical lines: ', len(cl), ' total lines=', len(g.grid.line))
print('Number of critical trafos: ', len(ct), ' total trafo=', len(g.grid.trafo))


Number of power flow:  122
Number of critical lines:  38  total lines= 75
Number of critical trafos:  40  total trafo= 53


Running contingency on all lines and trafo not realistic. Example some transformers/lines will be the only connection point to some loads, by taking these out-of-service they will of course be unsupplied and therefore fail our violation tests.

## Create base contingency scenario
By excluding all critical lines and trafos from the first run we get a list of contingency scenarios that will pass our violations check with an unchanged net. 

In [8]:
line_to_test = list(set(net.line.index)-set(cl))
trafo_to_test = list(set(net.trafo.index)-set(ct))

contingency_scenario = [line_to_test, trafo_to_test]

In [9]:
#Reset counter to track number of power flows performed for 1 contingnency analysis
analysis_check.check_violations.counter=0

#Perform contingency analysis
cl,ct =analysis_check.contingency_test(g.grid,vmax=contingency_limits['vmax'],
                           vmin=contingency_limits['vmin'],
                           max_line_loading=contingency_limits['max_line_loading'],
                           max_trafo_loading=contingency_limits['max_trafo_loading'],
                           p_lim=contingency_limits['subscription_p_limits'],
                           run_control=contingency_limits['run_controllers'],
                           contingency_scenario=contingency_scenario)


In [10]:
#print results
print('Number of power flow: ', analysis_check.check_violations.counter)
print('Number of critical lines: ', len(cl), ' total lines=', len(g.grid.line))
print('Number of critical trafos: ', len(ct), ' total trafo=', len(g.grid.trafo))


Number of power flow:  44
Number of critical lines:  0  total lines= 75
Number of critical trafos:  0  total trafo= 53


In [11]:
#Store data for further use
#s = json.dumps(line_to_test)
#open(parentdir+'\data\\svedala\\line_to_test.json', "w").write(s)
#s = json.dumps(trafo_to_test)
#open(parentdir+'\data\\svedala\\trafo_to_test.json', "w").write(s)